In [1]:
!nvidia-smi

Fri Apr 12 13:33:48 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.14                 Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3060       WDDM | 00000000:01:00.0  On |                  N/A |
|  0%   36C    P8               18W / 180W|   1196MiB / 12288MiB |      2%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
torch.cuda.is_available()

True

In [4]:
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
def get_bounding_box(ground_truth_map):
    # get bounding box from mask
    y_indices, x_indices = np.where(ground_truth_map > 0)
    x_min, x_max = np.min(x_indices), np.max(x_indices)
    y_min, y_max = np.min(y_indices), np.max(y_indices)
    # add perturbation to bounding box coordinates
    H, W = ground_truth_map.shape
    x_min = max(0, x_min - np.random.randint(0, 20))
    x_max = min(W, x_max + np.random.randint(0, 20))
    y_min = max(0, y_min - np.random.randint(0, 20))
    y_max = min(H, y_max + np.random.randint(0, 20))
    bbox = [x_min, y_min, x_max, y_max]

    return bbox

In [7]:
from torch.utils.data import Dataset

class SAMDataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item["image"]
        ground_truth_mask = np.array(item["mask"])

        # get bounding box prompt
        prompt = get_bounding_box(ground_truth_mask)

        # prepare image and prompt for the model
        inputs = self.processor(image, input_boxes=[[prompt]], return_tensors="pt")

        # remove batch dimension which the processor adds by default
        inputs = {k:v.squeeze(0) for k,v in inputs.items()}

        # add ground truth segmentation
        inputs["ground_truth_mask"] = ground_truth_mask

        return inputs

In [3]:
pip install transformers

  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     -------------------------------------- 42.0/42.0 kB 675.2 kB/s eta 0:00:00
Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
   ---------------------------------------- 0.0/388.9 kB ? eta -:--:--
   --------------------------------------- 388.9/388.9 kB 25.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.5 kB ? eta -:--:--
   --------------------------------------- 269.5/269.5 kB 16.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.5 kB ? eta -:--:--
   ------------------------------------- -- 256.0/269.5 kB 7.9 MB/s eta 0:00:01
   ---------------------------------------- 269.5/269.5 kB 5.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 70.4 MB/s eta 0:00:00
Note: you may need to restart the kernel

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
from transformers import SamProcessor

processor = SamProcessor.from_pretrained("facebook/sam-vit-base")

In [ ]:
def create_image_dataset(image_folder_path, mask_folder_path, target_size=(256, 256)):
    image_paths = []
    mask_paths = []
    
    for image_file in os.listdir(image_folder_path):
        if image_file.endswith(".jpg") or image_file.endswith(".jpeg") or image_file.endswith(".png"):
            image_path = os.path.join(image_folder_path, image_file)
            mask_file = f"masked_image_{image_file}"
            mask_path = os.path.join(mask_folder_path, mask_file)
            
            if os.path.isfile(mask_path):
                # 이미지와 마스크를 열고 리사이징
                image = Image.open(image_path).resize(target_size)
                mask = Image.open(mask_path).resize(target_size)
                
                image_paths.append(image)
                mask_paths.append(mask)
    
    dataset = Dataset.from_dict({
        'image': image_paths,
        'mask': mask_paths
    })
    
    return dataset

image_folder_path = './datasets/eye_image_seg_xl/labeled_data/masked_images'
mask_folder_path = './datasets/eye_image_seg_xl/labeled_data/masked_images'

dataset = create_image_dataset(image_folder_path, mask_folder_path, target_size=(256, 256))

print(dataset[0]['image'].size, dataset[0]['mask'].size)

In [8]:
train_dataset = SAMDataset(dataset=dataset, processor=processor)

NameError: name 'dataset' is not defined